In [2]:
import sqlite3 as lite
from monitoring import DB_PATH, SENSORS, SensorDB



In [9]:

import pandas as pd

con = lite.connect(f'file:{DB_PATH}?mode=ro', uri=True)
df = pd.read_sql_query("SELECT * from sensor_data", con)
con.close()

In [11]:
import plotly.express as px
#print(df)

fig = px.line(df, x="timestamp", y="value",)
fig.show()

    id    name            timestamp  value
0    1  window  2021-01-09 01:00:09  -2.00
1    2  window  2021-01-09 01:00:20  -2.00
2    3  window  2021-01-09 01:01:14  -2.00
3    4  window  2021-01-09 01:01:25 -10.00
4    5  window  2021-01-09 01:01:35  -2.00
5    6  window  2021-01-09 01:01:45  -2.00
6    7  window  2021-01-09 01:01:55  -2.00
7    8  window  2021-01-09 01:02:05  -2.00
8    9  window  2021-01-09 01:02:15  -2.00
9   10  window  2021-01-09 01:02:26  -2.00
10  11  window  2021-01-09 01:02:36 -10.00
11  12  window  2021-01-09 01:02:46  -2.00
12  13  window  2021-01-09 01:02:56  -2.00
13  14  window  2021-01-09 01:03:06  -2.00
14  15  window  2021-01-09 01:03:17  -2.00
15  16  window  2021-01-09 01:03:27  -2.00
16  17  window  2021-01-09 01:03:37  -2.00
17  18  window  2021-01-09 01:03:47  -2.00
18  19  window  2021-01-09 01:03:58 -10.00
19  20  window  2021-01-09 01:04:08  -2.00
20  21  window  2021-01-09 01:04:18  -2.00
21  22  window  2021-01-09 01:04:28  -2.00
22  23  win